In [1]:
import matplotlib.pyplot as plt
import os 
import shutil

In [2]:
classes=os.listdir("IMAGE")

In [3]:
classes

['Aerodactyl',
 'Bulbasaur',
 'Charmander',
 'Dratini',
 'Fearow',
 'Meowth',
 'Pikachu',
 'Psyduck',
 'Spearow',
 'Squirtle']

In [4]:
for i in classes:
    images=os.listdir(os.path.join("./IMAGE",i))
    print(i,len(images))

Aerodactyl 80
Bulbasaur 255
Charmander 264
Dratini 98
Fearow 111
Meowth 63
Pikachu 268
Psyduck 143
Spearow 117
Squirtle 248


## Now make new directory and transfer 10% of images of each category to val/category

In [5]:
os.mkdir("val")

In [6]:
for i in classes:
    images=os.listdir(os.path.join("./IMAGE",i))
    length=int(.1*len(images))
    os.mkdir("val/"+i)
    for j in images[:length]:
        shutil.move("./IMAGE/"+i+"/"+j,"./val/"+i)

In [7]:
for i in classes:
    images1=os.listdir(os.path.join("./IMAGE",i))
    print("train",i,len(images1))
    images2=os.listdir(os.path.join("./val",i))
    print("val",i,len(images2))

train Aerodactyl 72
val Aerodactyl 8
train Bulbasaur 230
val Bulbasaur 25
train Charmander 238
val Charmander 26
train Dratini 89
val Dratini 9
train Fearow 100
val Fearow 11
train Meowth 57
val Meowth 6
train Pikachu 242
val Pikachu 26
train Psyduck 129
val Psyduck 14
train Spearow 106
val Spearow 11
train Squirtle 224
val Squirtle 24


In [8]:
from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet import preprocess_input

Using TensorFlow backend.
C:\Users\Aditya Rayer\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Aditya Rayer\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Aditya Rayer\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Aditya Rayer\anaconda3\lib\site-packages\tensorflo

In [11]:
model=MobileNet(weights="imagenet",input_shape=(224,224,3),include_top=False)

In [12]:
model.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32

In [13]:
from keras.preprocessing.image import ImageDataGenerator

In [14]:
train=ImageDataGenerator(preprocessing_function=preprocess_input)
val=ImageDataGenerator(preprocessing_function=preprocess_input)

In [15]:
tr_gen=train.flow_from_directory("IMAGE/",target_size=(224,224),batch_size=32,class_mode="categorical")

Found 1482 images belonging to 10 classes.


In [16]:
val_gen=val.flow_from_directory("val/",target_size=(224,224),batch_size=32,class_mode="categorical")

Found 160 images belonging to 10 classes.


## Now we have a model whose dense layer have been removed now add layer using functional api

In [17]:
from keras.models import Model
from keras.layers import *

In [18]:
model.output

<tf.Tensor 'conv_pw_13_relu_1/Relu6:0' shape=(?, 7, 7, 1024) dtype=float32>

In [28]:
avl=GlobalAveragePooling2D()(model.output)
fc1=Dense(256,activation="relu")(avl)
d1=Dropout(0.5)(fc1)
fc2=Dense(10,activation="softmax")(d1)

In [29]:
complete_model=Model(model.input,fc2)

In [30]:
complete_model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [31]:
for ix in range(88):
    complete_model.layers[ix].trainable=False

In [32]:

complete_model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [33]:
history=complete_model.fit_generator(tr_gen,epochs=5,steps_per_epoch=50,validation_data=val_gen,validation_steps=5)

Epoch 1/5


C:\Users\Aditya Rayer\anaconda3\lib\site-packages\PIL\Image.py:961: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


50/50 [==============================] - 89s 2s/step - loss: 1.4708 - accuracy: 0.5353 - val_loss: 1.0499 - val_accuracy: 0.7000
Epoch 2/5
50/50 [==============================] - 86s 2s/step - loss: 0.6422 - accuracy: 0.7925 - val_loss: 0.6502 - val_accuracy: 0.7063
Epoch 3/5
50/50 [==============================] - 89s 2s/step - loss: 0.4638 - accuracy: 0.8492 - val_loss: 0.5467 - val_accuracy: 0.7812
Epoch 4/5
50/50 [==============================] - 80s 2s/step - loss: 0.3873 - accuracy: 0.8739 - val_loss: 0.6925 - val_accuracy: 0.8313
Epoch 5/5
50/50 [==============================] - 80s 2s/step - loss: 0.3065 - accuracy: 0.9010 - val_loss: 0.9888 - val_accuracy: 0.7875


In [34]:
import pandas as pd
import numpy as np
from keras.preprocessing import image

In [42]:
data=pd.read_csv("sample_submission.csv")

In [44]:
for idx,i in enumerate(data["Name"]):
    img_path ="./test/"+i
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    
    t=np.argmax(complete_model.predict(x))
    data.iloc[idx,1]=t

In [45]:
data.to_csv("fin.csv",index=False)

In [46]:
val_gen.class_indices,tr_gen.class_indices

({'Aerodactyl': 0,
  'Bulbasaur': 1,
  'Charmander': 2,
  'Dratini': 3,
  'Fearow': 4,
  'Meowth': 5,
  'Pikachu': 6,
  'Psyduck': 7,
  'Spearow': 8,
  'Squirtle': 9},
 {'Aerodactyl': 0,
  'Bulbasaur': 1,
  'Charmander': 2,
  'Dratini': 3,
  'Fearow': 4,
  'Meowth': 5,
  'Pikachu': 6,
  'Psyduck': 7,
  'Spearow': 8,
  'Squirtle': 9})